# Pettingzoo inside ray

In [1]:
from pettingzoo.mpe import simple_tag_v2
from pettingzoo.test import render_test
from pettingzoo.test import performance_benchmark
from pettingzoo.test import test_save_obs
import ray

import time
import random

In [8]:
cycles = 1
agent_count = 4
current_cycle = 0
action_queue = []

env = simple_tag_v2.env(render_mode="rgb_array", max_cycles=cycles)
env.reset()
for agent in env.agent_iter():
    if current_cycle >= cycles * agent_count:
        break
    if current_cycle % agent_count == 0:
        adversary_0_action = random.choice([0, 1, 2, 3, 4])
        adversary_1_action = random.choice([0, 1, 2, 3, 4])
        adversary_2_action = random.choice([0, 1, 2, 3, 4])
        good_agent_action = random.choice([0, 1, 2, 3, 4])

        action_queue += [
            adversary_0_action,
            adversary_1_action,
            adversary_2_action,
            good_agent_action
        ]
    print(agent)
    env.render()
    # obs, reward, done, info = env.last()
    observation, cumulative_rewards, terminations, truncations, infos = env.last()
    print(observation, cumulative_rewards, terminations, truncations, infos)
    action = action_queue.pop(0)
    env.step(action)
    current_cycle += 1
else:
    env.close()

adversary_0
[ 0.          0.         -0.81671894 -0.35830274  0.7397504   0.87728226
  0.24053054 -0.3338143   0.4263175  -0.13701786  1.0103662  -0.5890981
  0.5039304   0.7280103   0.          0.        ] 0.0 False False {}
adversary_1
[ 0.          0.         -0.39040145 -0.4953206   0.3134329   1.0143001
 -0.18578696 -0.19679645 -0.4263175   0.13701786  0.58404875 -0.45208025
  0.07761288  0.86502814  0.          0.        ] 0.0 False False {}
adversary_2
[ 0.          0.          0.19364728 -0.94740087 -0.27061585  1.4663804
 -0.7698357   0.2552838  -1.0103662   0.5890981  -0.58404875  0.45208025
 -0.5064359   1.3171084   0.          0.        ] 0.0 False False {}
agent_0
[ 0.          0.         -0.31278858  0.36970752  0.23582003  0.14927197
 -0.26339984 -1.0618246  -0.5039304  -0.7280103  -0.07761288 -0.86502814
  0.5064359  -1.3171084 ] 0.0 False False {}


## info

`observation, cumulative_rewards, terminations, truncations, infos = env.last()`

examples of the above returned values:

`observation = [ 0.          0.         -0.81671894 -0.35830274  0.7397504   0.87728226  0.24053054 -0.3338143   0.4263175 -0.13701786  1.0103662  -0.5890981 0.5039304   0.7280103   0.          0.        ]`

observations are ordered like this:

`[self_vel, self_pos, landmark_rel_positions[..], other_agent_rel_positions[..], other_agent_velocities[..]]`

`cumulative_rewards = 0.0`

`terminations = False `

`truncations = False `

`infos = {}`


then for agent names and actions, they are:


`agent names: adversary_0, adversary_1 ..., agent_0`

`actions: 0, 1, 2, 3 or 4 (meaning no_action, move_left, move_right, move_down, move_up)`

# Original testing code

In [1]:
# Approximate pi using random sampling. Generate x and y randomly between 0 and 1. 
#  if x^2 + y^2 < 1 it's inside the quarter circle. x 4 to get pi. 
import ray
from random import random

In [2]:

# Let's start Ray
ray.init()

SAMPLES = 1000000; 
# By adding the `@ray.remote` decorator, a regular Python function
# becomes a Ray remote function.
@ray.remote
def pi4_sample():
    in_count = 0
    for _ in range(SAMPLES):
        x, y = random(), random()
        if x*x + y*y <= 1:
            in_count += 1
    return in_count

# To invoke this remote function, use the `remote` method.
# This will immediately return an object ref (a future) and then create
# a task that will be executed on a worker process. Get retreives the result. 
future = pi4_sample.remote()
pi = ray.get(future) * 4.0 / SAMPLES
print(f'{pi} is an approximation of pi') 

# Now let's do this 100,000 times. 
# With regular python this would take 11 hours
# Ray on a modern laptop, roughly 2 hours
# On a 10-node Ray cluster, roughly 10 minutes 
BATCHES = 100000
results = [] 
for _ in range(BATCHES):
    results.append(pi4_sample.remote())
output = ray.get(results)
pi = sum(output) * 4.0 / BATCHES / SAMPLES
print(f'{pi} is a way better approximation of pi') 

2023-03-10 19:06:51,306	INFO worker.py:1553 -- Started a local Ray instance.


3.141644 is an approximation of pi


KeyboardInterrupt: 

In [3]:
for i in results:
    ray.cancel(i) 

2023-03-10 19:07:21,608	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=9720, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 11, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(4845913810e67b7effffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:07:21,936	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=26092, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
 

2023-03-10 19:07:23,889	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=4372, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 11, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(11e8eb0b985ea3eeffffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:07:23,919	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=25644, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
 

2023-03-10 19:07:26,771	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=2072, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 11, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(413e394227112558ffffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:07:27,778	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=26092, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
 

2023-03-10 19:07:35,001	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=29248, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 11, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(3bc0f941dca47a76ffffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:07:35,998	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=25644, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler


2023-03-10 19:07:38,396	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=23728, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 11, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(84f612c1e8cb7e05ffffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:07:38,424	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=25644, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler


...

2023-03-10 19:07:53,556	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=2072, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 11, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(7a66ffa7e7ae63ceffffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:07:53,657	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=22484, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
 

2023-03-10 19:07:55,782	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=2072, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 13, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(dccbb5abc0ffb12dffffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:07:55,835	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=29248, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
 

2023-03-10 19:07:56,433	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=25644, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 11, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(3ec0e711793102f5ffffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:07:56,479	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=9720, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
 

2023-03-10 19:07:56,991	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=2072, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 12, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(e4968836ae03d4baffffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:07:59,018	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=29248, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
 

2023-03-10 19:08:04,048	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=4372, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  File "python\ray\_raylet.pyx", line 805, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 850, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 857, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 861, in ray._raylet.execute_task
  File "C:\Users\rober\AppData\Local\Temp\ipykernel_32404\898370870.py", line 13, in pi4_sample
ray.exceptions.TaskCancelledError: Task: TaskID(8e9a10c87c4f5626ffffffffffffffffffffffff01000000) was cancelled
2023-03-10 19:08:04,051	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::pi4_sample() (pid=2072, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1072, in ray._raylet.execute_task_with_cancellation_handler
  